# Библиотеки

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.io import to_image
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
import seaborn as sn
import plotly.graph_objects as go
from random import randint

In [ ]:
#pip install --upgrade plotly

In [ ]:
#pip install --upgrade sympy

In [ ]:
pip install -U kaleido

# Система с внутривидовой конкуренцией

# Функции служебные

### График plotly

In [ ]:
# График plotly: Evolution of species with competition for resources
def draw_integ_evol(dataframe, Smax=""):
    if "Smax" in dataframe.columns:
      Smax = dataframe.Smax[0]
      dataframe = dataframe.drop("Smax", axis=1)

    df = dataframe.rename(columns={
        "S": "S (Ресурс)",
        "N1": "N1 (Вид 1)",
        "N2": "N2 (Вид 2)"

    })
    fig = px.line(df, x=df.index, y=["N1 (Вид 1)", "N2 (Вид 2)"],
                  width=1000, height=600,
                  color_discrete_map={
                      "N1 (Вид 1)": "blue",
                      "N2 (Вид 2)": "green"
                  })
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df["S (Ресурс)"],
            name="S (Ресурс)",
            line=dict(color="red"),
            yaxis="y2"
        )
    )
    fig.update_layout(
        title= f'Динамика численности видов и ресурса при Smax={Smax}',
        margin=dict(l=50, r=250, t=50, b=50),
        xaxis_title="Время",
        yaxis_title="Численность видов",
        yaxis2=dict(
          title="Количество ресурса",
          overlaying="y",
          side="right",
          showgrid=False
        ),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        xaxis=dict(
            ticks='outside',
            showline=True,
            linecolor='lightgray',
            # mirror=True,
            gridcolor='lightgray',
            showgrid=False
        ),
        yaxis=dict(
            gridcolor='lightgray',
            gridwidth=1,
            showgrid=True
        ),
        legend=dict(
            title="Ресурсы и виды",
            orientation="v",
            font=dict(size=8),
            yanchor="middle",
            y=0.9,
            xanchor="left",
            x=1.05
        )
    )
    fig.show()
    path = f"result_Smax={Smax}.svg"
    fig.write_image(path, engine="kaleido")

### Построение таблицы для для интегрирования с остановкой

In [ ]:
def make_df_evol(integr_result, t_list, column_names=[]):
  sol = np.array(integr_result).T
  dict = {}
  if len(column_names) > 0:
    df = pd.DataFrame(integr_result, columns=column_names[1:(len(sol)+1)])
    df.index = t_list
  else:
    for i in range(0, len(sol)):
      switch = (len(sol)+1)//2
      if i < switch:
        key = "S"+ str(i+1)
      else:
        key = "N"+ str(i - switch +1)
      dict[key] = sol[i]
      df = pd.DataFrame.from_dict(dict)
      df.index = t_list
  return(df)

### Интегрирования

In [ ]:
def integration_to_eps_many(dX_dt, t0, t1, dt, X0, K1, K2, a1, a2, c1, c2, delta,\
                            epsilon1, epsilon2, Smax, epsil=0.00005):
    t_eval = np.arange(t0, t1, dt)
    params = (K1, K2, a1, a2, c1, c2, delta, epsilon1, epsilon2, Smax)
    sol = solve_ivp(dX_dt, [t0, t1], X0, args=params, t_eval=t_eval, atol=epsil, rtol=epsil)
    integr_result = sol.y.T
    t_final = sol.t

    for i in range(1000, len(integr_result)):
        if abs(np.sum(integr_result[i]) - np.sum(np.mean(integr_result[i-1000:i], axis=0))) > epsil:
            continue
        else:
            return integr_result[:i+1], t_final[:i+1]

    return integr_result, t_final

### Весь цикл интегрирования

In [ ]:
# Весь анализ для чтения из таблицы: подготовка данных, график,
# интегрирование - таблицы с полным результатом и последними значениями.
def whole_cycle_table(data_set_table, dX_dt, t0, t1, dt, epsil=0.00005):
  full_integration_set = []
  last_stage_integration_set = []
  column_names = ['Smax', 'S', 'N1', 'N2']
  combined_last_stage_integration = []
  for i in range(0, len(data_set_table)):
    Smax = full_data.iloc[i]['Smax']
    delta = full_data.iloc[i]['delta']
    epsilon1 = full_data.iloc[i]['epsilon1']
    epsilon2 = full_data.iloc[i]['epsilon2']
    a1 = full_data.iloc[i]['a1']
    a2 = full_data.iloc[i]['a2']
    K1 = full_data.iloc[i]['K1']
    K2 = full_data.iloc[i]['K2']
    c1 = full_data.iloc[i]['c1']
    c2 = full_data.iloc[i]['c2']
    integr_result, t_final = integration_to_eps_many(dX_dt, t0, t1, dt, X, K1,\
                                                     K2, a1, a2, c1, c2, delta, \
                                                     epsilon1, epsilon2, Smax, \
                                                     epsil)
    data_one_step_res = make_df_evol(integr_result, t_final, column_names)
    full_integration_set.append(data_one_step_res)
    data_one_step_res.insert(0, 'Smax', Smax)
    data_one_step_res = data_one_step_res.reset_index(names='t')
    if len(combined_last_stage_integration) < 1:
      combined_last_stage_integration = data_one_step_res.tail(1)
    else:
      combined_last_stage_integration = pd.concat( \
       [combined_last_stage_integration, data_one_step_res.tail(1)], axis=0)\
       .reset_index(drop=True)
  return full_integration_set, combined_last_stage_integration

##   Уравнения для интегрирования

In [ ]:
def dX_dt(t, X, K1, K2, a1, a2, c1, c2, delta, epsilon1, epsilon2, Smax):

  S = X[0]
  N1 = X[1]
  N2 = X[2]


  dS_dt = Smax  - ((a1 * N1 * S / (K1 + S)) + (a2 * N2 * S / (K2 + S)) + (S / delta))
  dN1_dt = a1 * N1 * S / (K1 + S) - c1 * N1 - epsilon1 * (N1**2)
  dN2_dt = a2 * N2 * S / (K2 + S) - c2 * N2 - epsilon2 * (N2**2)

  return np.array([dS_dt, dN1_dt, dN2_dt])

# Симуляция

### Создание датасета для симуляции

In [ ]:
n = 40
Smax = [randint(10, 300) for p in range(0, n)]
delta = [0.2] * n
epsilon1 = [0.01] * n
epsilon = [0.01] * n
a1 = [1.5] * n
a2 = [1.5] * n
K1 = [10] * n
K2 = [10] * n
c1 = [0.6] * n
c2 = [0.6] * n
S = 10
N1 = 10
N2 = 10
full_data = pd.DataFrame(
    {'Smax': Smax,
     'S': S,
     'N1': N1,
     'N2': N2,
     'delta': delta,
     'epsilon1': epsilon1,
     'epsilon2': epsilon,
     'a1': a1,
     'a2': a2,
     'K1': K1,
     'K2': K2,
     'c1': c1,
     'c2': c2,
    })
t0 = 0
t1 = 500
dt = 0.001
t = np.linspace(t0, t1, 10000)
X = np.concatenate((S, N1, N2), axis=None)

In [ ]:
# full_integration_set, combined_last_stage_integration =  whole_cycle_table(full_data, dX_dt, t0, t1, dt, epsil=0.00001)

In [ ]:
# full_integration_set[0]

## Матрица корреляций

In [ ]:
# corel_matrix = combined_last_stage_integration[combined_last_stage_integration.columns[1:]].corr()

In [ ]:
# plt.figure(figsize=(8,8))
# sn.heatmap(corel_matrix, vmin=-0.1, vmax=1, cmap="flare", annot=True)
# plt.show()

# Стационарные точки и анализ их устойчивости по Ляпунову

## Тривиальное равновесие: N1 = 0, N2 = 0

In [ ]:
#Тривиальное равновесие
Smax = 10
delta = 0.2
epsilon1 = 0.01
epsilon2 = 0.01
a1 = 0.3
a2 = 1.6
K1 = 5
K2 = 27
c1 = 0.1
c2 = 0.6

S = Smax * delta
N1 = 0
N2 = 0

print(f"S = {round(S, 2)}")
print(f"N1 = {round(N1, 2)}")
print(f"N2 = {round(N2, 2)}")

lambda1 = -1/delta
lambda2 = (a1 * Smax * delta) / (K1 + Smax * delta) - c1
lambda3 = (a2 * Smax * delta) / (K2 + Smax * delta) - c2

print(f"lambda1 = {round(lambda1, 2)}", "lambda1 < 0:", lambda1<0)
print(f"lambda2 = {round(lambda2, 2)}","lambda2 < 0:", lambda2<0)
print(f"lambda3 = {round(lambda3, 2)}","lambda3 < 0:", lambda3<0)

S = 2.0
N1 = 0
N2 = 0
lambda1 = -5.0 lambda1 < 0: True
lambda2 = -0.01 lambda2 < 0: True
lambda3 = -0.49 lambda3 < 0: True


In [ ]:
S, N1, N2 = 10, 10, 10
t0 = 0
t1 = 300
dt = 0.001
X0 = np.concatenate((S, N1, N2), axis=None)
integr_result, t_final = integration_to_eps_many(dX_dt, t0, t1, dt, X0, K1, K2, a1, a2, c1, c2, delta, epsilon1, epsilon2, Smax, epsil=0.000001)

In [ ]:
df = make_df_evol(integr_result, t_final, ["t", "S", "N1", "N2"])
draw_integ_evol(df, Smax)

## Smax = 40, N1 > 0, N2 = 0

In [ ]:
#N1>0 N2=0
Smax = 40
delta = 0.2
epsilon1 = 0.01
epsilon2 = 0.01
a1 = 0.3
a2 = 1.6
K1 = 5
K2 = 27
c1 = 0.1
c2 = 0.6

def equation(S):
    return Smax - ((a1**2) * (S**2)) / (epsilon1 * ((K1 + S)**2)) + (a1 * c1 * S) / (epsilon1 * (K1 + S)) - S / delta

initial_guess = 10
S  = fsolve(equation, initial_guess)[0]
print(f"S = {round(S, 2)}")

N1 = ((a1 * S) / (K1 + S) - c1) / epsilon1
N2 = 0
print(f"N1 = {round(N1, 2)}")
print(f"N2 = {round(N2, 2)}")

lambda1 = - (a1 * N1 * K1 / (K1 + S)**2) - 1/delta
lambda2 = c1 - (a1 * S) / (K1 + S)
lambda3 = (a2 * S) / (K2 + S) - c2

print(f"lambda1 = {round(lambda1, 2)}", "lambda1 < 0:", lambda1<0)
print(f"lambda2 = {round(lambda2, 2)}","lambda2 < 0:", lambda2<0)
print(f"lambda3 = {round(lambda3, 2)}","lambda3 < 0:", lambda3<0)


S = 7.7
N1 = 8.19
N2 = 0
lambda1 = -5.08 lambda1 < 0: True
lambda2 = -0.08 lambda2 < 0: True
lambda3 = -0.24 lambda3 < 0: True


In [ ]:
S, N1, N2 = 10, 10, 10
t0 = 0
t1 = 200
dt = 0.001
X0 = np.concatenate((S, N1, N2), axis=None)
integr_result, t_final = integration_to_eps_many(dX_dt, t0, t1, dt, X0, K1, K2, a1, a2, c1, c2, delta, epsilon1, epsilon2, Smax, epsil=0.000001)

In [ ]:
df = make_df_evol(integr_result, t_final, ["t", "S", "N1", "N2"])
draw_integ_evol(df, Smax)

## Smax = 150, N1 > 0, N2 > 0

In [ ]:
Smax = 150
delta = 0.2
epsilon1 = 0.01
epsilon2 = 0.01
a1 = 0.3
a2 = 1.6
K1 = 5
K2 = 27
c1 = 0.1
c2 = 0.6

# Определяем функцию для уравнения
def equation(S):
    return Smax - ((a1**2) * (S**2)) / (epsilon1 * ((K1 + S)**2)) \
    + (a1 * c1 * S) / (epsilon1 * (K1 + S)) \
    - ((a2**2) * (S**2)) / (epsilon2 * ((K2 + S)**2))\
    + (a2 * c2 * S) / (epsilon2 * (K2 + S)) \
    - S / delta

# Начальное приближение для S
initial_guess = 10

# Решаем уравнение
S  = fsolve(equation, initial_guess)[0]
print(f"S = {round(S, 2)}")

N1 = ((a1 * S) / (K1 +S) - c1) / epsilon1
N2 = ((a2 * S) / (K2 +S) - c2) / epsilon2
print(f"N1 = {round(N1, 2)}")
print(f"N2 = {round(N2, 2)}")


J11 = -(a1 * N1 * K1) / (K1 + S)**2 - (a2 * N2 * K2) / (K2 + S)**2 - 1 / delta
J12 = -(a1 * S) / (K1 + S)
J13 = -(a2 * S) / (K2 + S)
J21 = (a1 * N1 * K1) / (K1 + S)**2
J22 = (a1 * S) / (K1 + S) - c1 - 2 * epsilon1 * N1
J31 = (a2 * N2 * K2) / (K2 + S)**2
J33 = (a2 * S) / (K2 + S) - c2 - 2 * epsilon2 * N2

# Матрица Якоби
J = np.array([
    [J11, J12, J13],
    [J21, J22, 0],
    [J31, 0, J33]
])

# Вычисление собственных значений
eigenvalues, _ = np.linalg.eig(J)
print()

lambda1 = eigenvalues[0]
lambda2 = eigenvalues[1]
lambda3 = eigenvalues[2]

print(f"lambda1 = {round(lambda1, 2)}", "lambda1 < 0:", lambda1<0)
print(f"lambda2 = {round(lambda2, 2)}","lambda2 < 0:", lambda2<0)
print(f"lambda3 = {round(lambda3, 2)}","lambda3 < 0:", lambda3<0)


S = 26.26
N1 = 15.2
N2 = 18.88

lambda1 = -5.27 lambda1 < 0: True
lambda2 = -0.23 lambda2 < 0: True
lambda3 = -0.15 lambda3 < 0: True


In [ ]:
S, N1, N2 = 10, 10, 10
t0 = 0
t1 = 200
dt = 0.001
X0 = np.concatenate((S, N1, N2), axis=None)
integr_result, t_final = integration_to_eps_many(dX_dt, t0, t1, dt, X0, K1, K2, a1, a2, c1, c2, delta, epsilon1, epsilon2, Smax, epsil=0.000001)

In [ ]:
df = make_df_evol(integr_result, t_final, ["t", "S", "N1", "N2"])
draw_integ_evol(df, Smax)

## Smax = 1500, N1 > 0, N2 > 0

In [ ]:
Smax = 1500
delta = 0.2
epsilon1 = 0.01
epsilon2 = 0.01
a1 = 0.3
a2 = 1.6
K1 = 5
K2 = 27
c1 = 0.1
c2 = 0.6

# Определяем функцию для уравнения
def equation(S):
    return Smax - ((a1**2) * (S**2)) / (epsilon1 * ((K1 + S)**2)) \
    + (a1 * c1 * S) / (epsilon1 * (K1 + S)) \
    - ((a2**2) * (S**2)) / (epsilon2 * ((K2 + S)**2))\
    + (a2 * c2 * S) / (epsilon2 * (K2 + S)) \
    - S / delta

# Начальное приближение для S
initial_guess = 10

# Решаем уравнение
S  = fsolve(equation, initial_guess)[0]
print(f"S = {round(S, 2)}")

N1 = ((a1 * S) / (K1 +S) - c1) / epsilon1
N2 = ((a2 * S) / (K2 +S) - c2) / epsilon2
print(f"N1 = {round(N1, 2)}")
print(f"N2 = {round(N2, 2)}")


J11 = -(a1 * N1 * K1) / (K1 + S)**2 - (a2 * N2 * K2) / (K2 + S)**2 - 1 / delta
J12 = -(a1 * S) / (K1 + S)
J13 = -(a2 * S) / (K2 + S)
J21 = (a1 * N1 * K1) / (K1 + S)**2
J22 = (a1 * S) / (K1 + S) - c1 - 2 * epsilon1 * N1
J31 = (a2 * N2 * K2) / (K2 + S)**2
J33 = (a2 * S) / (K2 + S) - c2 - 2 * epsilon2 * N2

# Матрица Якоби
J = np.array([
    [J11, J12, J13],
    [J21, J22, 0],
    [J31, 0, J33]
])

# Вычисление собственных значений
eigenvalues, _ = np.linalg.eig(J)
print()

lambda1 = eigenvalues[0]
lambda2 = eigenvalues[1]
lambda3 = eigenvalues[2]

print(f"lambda1 = {round(lambda1, 2)}", "lambda1 < 0:", lambda1<0)
print(f"lambda2 = {round(lambda2, 2)}","lambda2 < 0:", lambda2<0)
print(f"lambda3 = {round(lambda3, 2)}","lambda3 < 0:", lambda3<0)


S = 273.91
N1 = 19.46
N2 = 85.64

lambda1 = -5.03 lambda1 < 0: True
lambda2 = -0.87 lambda2 < 0: True
lambda3 = -0.19 lambda3 < 0: True


In [ ]:
S, N1, N2 = 10, 10, 10
t0 = 0
t1 = 200
dt = 0.001
X0 = np.concatenate((S, N1, N2), axis=None)
integr_result, t_final = integration_to_eps_many(dX_dt, t0, t1, dt, X0, K1, K2, a1, a2, c1, c2, delta, epsilon1, epsilon2, Smax, epsil=0.000001)

In [ ]:
df = make_df_evol(integr_result, t_final, ["t", "S", "N1", "N2"])
draw_integ_evol(df, Smax)